In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, mean, stddev
from pyspark.sql.window import Window

# Replace these values with your actual storage account and key
storage_account_name = "harsh98"
container_name = "raw"
file_name = "healthcare.csv"
storage_account_key = "AV44REzH5Qj3cRLTllGrBC4D0DDKQuZcJNdtPokAAvVn213iylDKhdGkMPTCLt5WNMpaxjic1XlY+AStYoN9xQ==" 

# Set up the full path to the file in ADLS Gen2
file_path = f"abfss://raw@harsh98.dfs.core.windows.net/healthcare.csv"

# Initialize Spark session
spark = SparkSession.builder.appName("DataCleaning").getOrCreate()

# Set the configuration to use your Azure Storage account key
spark.conf.set(f"fs.azure.account.key.harsh98.blob.core.windows.net", "AV44REzH5Qj3cRLTllGrBC4D0DDKQuZcJNdtPokAAvVn213iylDKhdGkMPTCLt5WNMpaxjic1XlY+AStYoN9xQ==")

# Read the Delta file from ADLS Gen2
df = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load(file_path)

# Handling missing values
df = df.na.fill({
    "Age": df.agg(mean(col("Age"))).first()[0],
    "Gender": "Unknown",
    "Blood Type": "Unknown",
    "Medical Condition": "Unknown",
    "Doctor": "Unknown",
    "Hospital": "Unknown",
    "Insurance Provider": "Unknown",
    "Billing Amount": 0.0,
    "Room Number": "Unknown",
    "Admission Type": "Unknown",
    "Medication": "Unknown",
    "Test Results": "Unknown"
})

# Normalizing data
windowSpec = Window.partitionBy()
df = df.withColumn("Age", (col("Age") - mean(col("Age")).over(windowSpec)) / stddev(col("Age")).over(windowSpec))
df = df.withColumn("Billing Amount", (col("Billing Amount") - mean(col("Billing Amount")).over(windowSpec)) / stddev(col("Billing Amount")).over(windowSpec))

# Additional transformations can be added here

# Write the cleaned data to a Parquet file in ADLS Gen2
output_path = f"abfss://processed@harsh91.dfs.core.windows.net/cleaned_healthcare.parquet"

In [0]:
df.show(5)

+-------------------+--------------------+------+----------+-----------------+-----------------+--------------+--------------------+------------------+-------------------+-----------+--------------+--------------+-----------+------------+
|               Name|                 Age|Gender|Blood Type|Medical Condition|Date of Admission|        Doctor|            Hospital|Insurance Provider|     Billing Amount|Room Number|Admission Type|Discharge Date| Medication|Test Results|
+-------------------+--------------------+------+----------+-----------------+-----------------+--------------+--------------------+------------------+-------------------+-----------+--------------+--------------+-----------+------------+
|    Tiffany Ramirez|  1.5083893906507593|Female|        O-|         Diabetes|       2022-11-17|Patrick Parker|    Wallace-Hamilton|          Medicare| 0.8512068976809121|        146|      Elective|    2022-12-01|    Aspirin|Inconclusive|
|        Ruben Burns| -0.8398704449354746|  

In [0]:
df.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Gender: string (nullable = false)
 |-- Blood Type: string (nullable = false)
 |-- Medical Condition: string (nullable = false)
 |-- Date of Admission: date (nullable = true)
 |-- Doctor: string (nullable = false)
 |-- Hospital: string (nullable = false)
 |-- Insurance Provider: string (nullable = false)
 |-- Billing Amount: double (nullable = true)
 |-- Room Number: integer (nullable = true)
 |-- Admission Type: string (nullable = false)
 |-- Discharge Date: date (nullable = true)
 |-- Medication: string (nullable = false)
 |-- Test Results: string (nullable = false)

